# Baseline

This guide is a good beginning to familiarize with basic `mml` usage. The goal is to run some baseline models for an existing task. We will use the widely known `MNIST` dataset for demonstration purposes.

## Step 1: Prepare data

`MNIST` is already available in `mml` through the `mml-tasks` plugin, which might be installed through `pip` easily.


In [2]:
!pip show mml-tasks

Name: mml-tasks
Version: 0.6.0
Summary: This is the MML tasks plugin, providing dataset and task implementation for a bunch of datasets.
Home-page: https://git.dkfz.de/imsy/ise/mml
Author: Patrick Godau
Author-email: patrick.godau@dkfz-heidelberg.de
License: MIT
Location: /home/scholzpa/miniconda3/envs/mml-dev/lib/python3.10/site-packages
Editable project location: /home/scholzpa/Documents/development/github/mml/plugins/tasks
Requires: mml-core
Required-by: 


In [4]:
# available datasets
!mml-tasks

+------------------------------------------------------------+---------------------+---------------------------+-----------+-------------------------------+
|                            name                            |       dataset       |            type           | installed |            license            |
+------------------------------------------------------------+---------------------+---------------------------+-----------+-------------------------------+
|                  idle_action_recognition                   |     idle_action     |       classification      |    True   |          CC_BY_NC_4_0         |
|             bean_plant_disease_classification              |        ibean        |       classification      |    True   |              MIT              |
|              caltech101_object_classification              |      caltech101     |       classification      |    True   |            UNKNOWN            |
|               cifar10_object_classification             

In [5]:
# install data for MNIST (data already installed here, might take a couple of minutes depending on hardware and ethernet connection)
!mml create task_list=[mnist_digit_classification]

[2025-05-02 22:11:23,262][mml][INFO] - Started MML 1.0.4 on Python 3.10.17 with mode CREATE.
[2025-05-02 22:11:23,263][mml][INFO] - Plugins loaded: ['mml-tasks']
[2025-05-02 22:11:23,595][mml.core.scripts.schedulers.create_scheduler][INFO] - Skipping creation of task mnist_digit_classification because there already seems to be a RAW version of that.
[2025-05-02 22:11:23,596][mml][INFO] - MML init time was 0.0h 0.0m  0.33s.
[2025-05-02 22:11:23,600][mml.core.scripts.schedulers.create_scheduler][INFO] - Starting task creation!
[2025-05-02 22:11:23,600][mml.core.data_loading.file_manager][INFO] - A total of 0 paths have been created during this run.
[2025-05-02 22:11:23,601][mml.core.scripts.schedulers.base_scheduler][INFO] - Successfully finished all experiments!
[2025-05-02 22:11:23,601][mml][INFO] - MML run time was 0.0h 0.0m  0.00s.


In [4]:
# preprocess task
!mml pp task_list=[mnist_digit_classification] preprocessing=size224

/home/scholzpa/miniconda3/envs/mml/lib/python3.8/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
[2024-11-22 15:18:20,420][mml][INFO] - Started MML 0.14.2 on Python 3.8.12 with mode PP.
[2024-11-22 15:18:20,420][mml][INFO] - Plugins loaded: ['mml-tasks', 'mml-dimensionality', 'mml-similarity', 'mml-tags', 'mml-sql', 'mml-drive', 'mml-lsf', 'mml-suggest', 'mml-prevalences', 'mml-tf']
[2024-11-22 15:18:20,649][mml.core.scripts.schedulers.base_scheduler][INFO] - Executing after init hook: check_lsf_workers
[2024-11-22 15:18:20,650][mml_lsf.workers][INFO] - LSF cluster plugin detected local system, no changes made to the number of workers.
[2024-11-22 15:18:20,657][mml][INFO] - MML init time was 0.0h 0.0m  0.24s.
[2024-11-22 15:18:20,659][mml.core.scripts.schedulers.base_scheduler][INFO] - Preparing 

## Step 2: Use defaults for training

In [ ]:
# to run a single training
!mml train pivot.name=mnist_digit_classification preprocessing=size224 proj=demo_baseline
# afterwards inspect the training via running tensorboard and inspecting through the browser
!tensorboard --logdir /path/to/mml/results/demo_baseline

![image not found](../../../../docs/source/_static/tensorboard_example.png "Tensorboard view")

## Step 3: Optimizing hyperparameters

We will perform a simple grid search with a couple of architectures & augmentation strategies.

In [ ]:
# to run a hyperparameter search (here: grid search with 3 models x 3 augmentation strategies),
# to speed things up: no cross-validation, only 1 epoch, deactivated learning rate tuning
!mml train pivot.name=mnist_digit_classification preprocessing=size224 arch.name=resnet18,tiny_vit_21m_224.dist_in22k_ft_in1k,tf_efficientnet_b0.ns_jft_in1k augmentations=basic,randaugment,load_imagenet_aa proj=demo_grid trainer.max_epochs=1 tune.lr=false sampling.batch_size=100 mode.cv=false  mode.nested=False +hpo/sampler=grid --multirun

at the end it prints:

> [HYDRA] Best parameters: {'arch.name': 'tiny_vit_21m_224.dist_in22k_ft_in1k', 'augmentations': 'randaugment'}
> [HYDRA] Best value: 0.0211497992277145

In [ ]:
# use the best hyperparameters in a full training run (the use_best_params is the name of our search, a folder inside the proj folder, inside "hpo", i.e. MML_RESULTS_PATH/demo_grid/hpo/2024-12-03_12-28-46_362374)
!mml train proj=demo_grid use_best_params=2024-12-03_12-28-46_362374 pivot.name=mnist_digit_classification

[mml][INFO] - -------------------------------------------------------------------------------------
[mml][INFO] - Loaded hpo results from study 2024-12-03_12-28-46_362374 and merged 2 params into config.
[mml][INFO] -   > arch.name=tiny_vit_21m_224.dist_in22k_ft_in1k
[mml][INFO] -   > augmentations=randaugment
[mml][INFO] - -------------------------------------------------------------------------------------
[mml][INFO] - Started MML 0.14.2 on Python 3.10.9 with mode TRAIN.
...

For more complex optimization setups (e.g. multi node optimization) one may use the `mml-sql` plugin. See the plugin documentation for installation and setup instructions. 